In [1]:
from sklearn import datasets
import xgboost as xgb

iris = datasets.load_iris()
X = iris.data
y = iris.target

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [3]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [10]:
from sklearn.model_selection import GridSearchCV

# The gamma parameter can also help with controlling overfitting
clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=3)

grid.fit(X_train, Y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, va

In [12]:
print('Best parameters:',grid.best_params_)
print('Best Scores:',grid.best_score_)

Best parameters: {'colsample_bytree': 0.5, 'eta': 0.3, 'gamma': 0.2, 'max_depth': 3, 'min_child_weight': 3}
Best Scores: -0.2059878522064537


In [13]:
param = {
    'eta': 0.3,
    'gamma': 0.2,
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3,
    'min_child_weight': 3,
    'colsample_bytree': 0.5} 

steps = 20  # The number of training iterations

In [14]:
model = xgb.train(param, D_train, steps)

In [15]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 1.0
Recall = 1.0
Accuracy = 1.0


In [16]:
#Once your XGBoost model is trained, you can dump a human readable description of it into a text file
model.dump_model('dump.raw.txt')